In [1]:
import gdspy
import numpy as np
from element import *
from sketch import *

In [2]:
element = Geometry()
element.isInverted[2] = True
rect = gdspy.Rectangle((0,0), (10e3,5e3))
element.boolean(rect, 'or', 2)
circle = gdspy.Round((0, 5e3), 2e3)
element.boolean(circle, 'or', 1)
# element.move((1,1))
# element.rotate(90*np.pi/180)
element.addPort((5e3, 0), 270*np.pi/180)
port = Port((20e3, 10e3), 0)
element.getPorts()
print(element.ports)
element.mergeWithPort(port, 0)
print(element.ports)
element.draw()

Порт 0, координаты (5000.000000, 0.000000), угол 4.712389
(-20000.0, 0.0)


In [3]:
sketch = Sketch('helloWorld')

In [4]:
sketch.addGeometry(element)
sketch.draw()

### Обозначения слоев
0 - металл с копланарами

1 - подложка

2 - металл с чипами

3 - vias

4 - порты

5 - технический уровень

In [5]:
# rect = gdspy.Rectangle((0,0), (1,2))
# rect2 = gdspy.Rectangle((0,0), (0.5,1))
# print(type(gdspy.boolean(rect,rect2, 'not')))
# cell.add(gdspy.boolean(rect,rect2, 'not'))
# gdspy.LayoutViewer()